In [15]:
from lxml import etree
import pandas as pd
import sys 
from ast import literal_eval
from itertools import combinations, product
sys.path.append('../types/')
import pickle
import json
import numpy as np

# getting withinRepo table:
- reponame, username, filename[], totalFileAppearances, clonePairCount 


In [16]:
with open("./parsed/clone_frags.pkl", "rb") as f:
    clone_frags = pickle.load(f)

In [17]:
# Define a custom aggregation function
def aggregate_clone_frags(group):
    return [{
        'fileName': frag['fileName'], 
        'startline': frag['startline'], 
        'endline': frag['endline'], 
        'pcid': frag['pcid'],
    } for _, frag in group.iterrows()]

# Group and aggregate using the custom function
withinRepo_df = clone_frags.groupby(by=['bucketId', 'userName', 'repoName', 'similarity']).apply(aggregate_clone_frags).reset_index(name='cloneFragments').sort_values(by=['bucketId', 'userName', 'repoName'], ascending=True)


/tmp/ipykernel_9700/4237163285.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  withinRepo_df = clone_frags.groupby(by=['bucketId', 'userName', 'repoName', 'similarity']).apply(aggregate_clone_frags).reset_index(name='cloneFragments').sort_values(by=['bucketId', 'userName', 'repoName'], ascending=True)


In [18]:
withinRepo_df

,bucketId,userName,repoName,similarity,cloneFragments
0,1,abdulghaffaransari,AI-Chat-Bot-Using-PyTorch,74,"[{'fileName': 'processor.py', 'startline': 10,..."
1,1,koenry,Flask-PyTorch-Chatbot,74,"[{'fileName': 'chat.py', 'startline': 5, 'endl..."
2,2,DongKeon,PyTorch-VAD,70,"[{'fileName': 'train.py', 'startline': 20, 'en..."
3,2,IMAC-projects,NSRR-PyTorch,70,"[{'fileName': 'train.py', 'startline': 21, 'en..."
4,2,ModelBunker,StarNet-PyTorch,70,"[{'fileName': 'train.py', 'startline': 20, 'en..."
...,...,...,...,...,...
8192,3323,mcgoughlin,PyTorch-UNet,75,"[{'fileName': 'unet_modules.py', 'startline': ..."
8193,3324,jiaxiang-cheng,PyTorch-CNN-for-RUL-Prediction,80,"[{'fileName': 'load_data.py', 'startline': 5, ..."
8194,3325,VikHerr,DeepVis-PredDiff-PyTorch,93,"[{'fileName': 'main.py', 'startline': 57, 'end..."
8195,3326,ConnorJL,ProgGAN-PyTorch,77,"[{'fileName': 'ProgGAN.py', 'startline': 186, ..."


In [19]:
withinRepo_df['clonePairs'] = withinRepo_df['cloneFragments'].apply(lambda x: list(combinations(x, 2)))

In [20]:
def filter_out_same_file(clonePairs):
    return [(pair[0]['fileName'], pair[1]['fileName']) for pair in clonePairs if pair[0]['fileName'] != pair[1]['fileName']]

In [21]:
withinRepo_df['clonePairs'] = withinRepo_df['clonePairs'].apply(filter_out_same_file)

In [22]:
withinRepo_df = withinRepo_df[withinRepo_df['clonePairs'].apply(lambda x: len(x) > 0)]
withinRepo_df['clonePairCount'] = withinRepo_df['clonePairs'].apply(lambda x: len(x))

/tmp/ipykernel_9700/1674674859.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  withinRepo_df['clonePairCount'] = withinRepo_df['clonePairs'].apply(lambda x: len(x))


In [23]:
withinRepo_df = withinRepo_df[withinRepo_df['clonePairCount'] > 0]
withinRepo_df

,bucketId,userName,repoName,similarity,cloneFragments,clonePairs,clonePairCount
20,7,DetectionBLWX,FPN.pytorch,80,"[{'fileName': 'demo.py', 'startline': 21, 'end...","[(demo.py, test.py)]",1
21,7,DetectionBLWX,ssdetection,80,"[{'fileName': 'demo.py', 'startline': 19, 'end...","[(demo.py, test.py)]",1
40,11,BenjaminAm,PyTorch-Tutorial,70,"[{'fileName': 'untitled0.py', 'startline': 66,...","[(untitled0.py, Tutorial3CatsDogs.py), (untitl...",3
42,11,BoriaK,AutoEncoder_PyTorch,70,"[{'fileName': 'models.py', 'startline': 220, '...","[(models.py, models_from_Avi.py)]",1
44,11,CeasonCui,CNN_PyTorch,70,"[{'fileName': '_5_cnn_acc_check.py', 'startlin...","[(_5_cnn_acc_check.py, _4_cnn_fm.py), (_5_cnn_...",3
...,...,...,...,...,...,...,...
8186,3317,ptklx,learn_deep_from_torchvsion,87,"[{'fileName': 'train.py', 'startline': 182, 'e...","[(train.py, train_egg_dirty.py)]",1
8187,3318,ptklx,learn_deep_from_torchvsion,92,"[{'fileName': 'train.py', 'startline': 369, 'e...","[(train.py, train_egg_dirty.py)]",1
8189,3320,dreamplus1989,yolov3-plus_PyTorch,73,"[{'fileName': 'train_coco.py', 'startline': 76...","[(train_coco.py, train_voc.py)]",1
8190,3321,dreamplus1989,yolov3-plus_PyTorch,70,"[{'fileName': 'test_coco.py', 'startline': 63,...","[(test_coco.py, test_voc.py)]",1


In [24]:
withinRepo_df.loc[40].cloneFragments

[{'fileName': 'untitled0.py', 'startline': 66, 'endline': 83, 'pcid': '49812'},
 {'fileName': 'Tutorial3CatsDogs.py',
  'startline': 65,
  'endline': 79,
  'pcid': '49816'},
 {'fileName': 'Tutorial2MNIST.py',
  'startline': 38,
  'endline': 51,
  'pcid': '49808'}]

In [25]:
# def getClonePairs(row):
#     return {
#         "userName": row['userName'],
#         "repoName": row['repoName'],
#         "clonePairs": [
#             {
#                 x: row['fileDict'][x],
#                 y: row['fileDict'][y] 
#             } for x, y in row['clonePairsSet']
#         ]
#     }
# def getWithinRepoJSON(group):
#     return {
#         "withinRepos": group.apply(getClonePairs, axis=1).tolist()
#     }

# def default_handler(obj):
#     if isinstance(obj, np.integer):
#         return int(obj)
#     elif isinstance(obj, set):
#         return list(obj)
#     raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# withinRepoJSON = {}
# for bucketId, group in withinRepo_df.groupby(by='bucketId'):
#     # Pass both bucketId and the group to the function
#     withinRepoJSON[str(bucketId)] = getWithinRepoJSON(group)

# with open("./parsed/withinRepo.json", "w") as f:
#     f.write(json.dumps(withinRepoJSON, default=default_handler, indent=4))


In [26]:
withinRepo_df.to_pickle("./parsed/withinRepo.pkl")

In [27]:
# so acrossRepos could be
# {
#     "1": [
#         {
#             cloneFrags: [
#                 { obj1: {fileName: "file1", repoName: "repo1", userName: "user1", startline: 1, endline: 10, pcid: 1}, obj2: {fileName: "file2", repoName: "repo2", userName: "user2", startline: 1, endline: 10} }
#             ]
#         }
#     ],
#     clonePairCount: 1
#     sameUser: false
# }

# withinRepo could be:
# {
#     "1": [
#         {
#             cloneFrags: [
#                { obj1: {fileName: "file1", repoName: "repo1", userName: "user1", startline: 1, endline: 10}, obj2: {fileName: "file2", repoName: "repo1", userName: "user1", startline: 1, endline: 10} }